In [1]:
from experiments import UniformMeshes, discretize, continuous_coefficient_2d
import numpy as np
import pandas as pd
import torch
import os
import json
import pyamgx
import tqdm

/usr/local/lib/python3.12/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


AMGX version 2.5.0
Built on Dec  8 2025, 14:48:22
Compiled with CUDA Runtime 12.6, using CUDA driver 12.8
The AMGX_initialize_plugins API call is deprecated and can be safely removed.


In [2]:
mesh_family = UniformMeshes(d=2, m=11)

In [3]:
def test_config(cfg, problem, rep_setup=3, rep_solve=5):
    rsc = pyamgx.Resources().create_simple(cfg)

    A = pyamgx.Matrix().create(rsc)
    b = pyamgx.Vector().create(rsc)
    x = pyamgx.Vector().create(rsc)

    A.upload_CSR(problem.exact_form_matrix)
    b.upload(problem.load_vector)

    solver = pyamgx.Solver().create(rsc, cfg)

    setup_times = []
    for _ in range(rep_setup):
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)

        start.record()
        solver.setup(A)
        end.record()
        torch.cuda.synchronize()
        setup_times.append(start.elapsed_time(end) / 1000)

    solve_times = []
    for _ in range(rep_solve):
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)

        x.set_zero(n=problem.load_vector.shape[0], block_dim=1)
        start.record()
        solver.solve(b, x, zero_initial_guess=True)
        end.record()
        torch.cuda.synchronize()
        solve_times.append(start.elapsed_time(end) / 1000)

    x_vec = np.zeros_like(problem.load_vector)
    x.download(x_vec)
    residual = np.linalg.norm(problem.exact_form_matrix @ x_vec - problem.load_vector)

    A.destroy()
    b.destroy()
    x.destroy()
    solver.destroy()
    rsc.destroy()

    return {
        "setup_time": min(setup_times),
        "solve_time": min(solve_times),
        "residual": residual,
    }

In [4]:
def patch_config(cfg):
    cfg = cfg.copy()
    cfg["solver"]["print_grid_stats"] = 0
    cfg["solver"]["print_solve_stats"] = 0
    cfg["solver"]["obtain_timings"] = 0
    cfg["solver"]["convergence"] = "RELATIVE_INI"
    cfg["solver"]["tolerance"] = 1e-9
    cfg["solver"]["max_iters"] = 1000
    if "preconditioner" in cfg:
        cfg["preconditioner"]["print_grid_stats"] = 0
        cfg["preconditioner"]["print_solve_stats"] = 0
    if "preconditioner" in cfg["solver"]:
        cfg["solver"]["preconditioner"]["print_grid_stats"] = 0
        cfg["solver"]["preconditioner"]["print_solve_stats"] = 0
    return cfg

In [5]:
configs = {}
for filename in tqdm.tqdm(os.listdir("/workspace/AMGX/src/configs/")):
    if filename.endswith(".json"):
        with open(f"/workspace/AMGX/src/configs/{filename}") as f:
            cfg_json = patch_config(json.load(f))
            name = filename[:-5]
            if "GMRES" in cfg_json["solver"]["solver"]:
                cfg_json["solver"]["solver"] = "PCG"
                name += "__MOD_PCG"
            configs[name] = cfg_json

100%|██████████| 63/63 [00:00<00:00, 14281.76it/s]


In [6]:
double_precision_discrete_problem = discretize(
    continuous_coefficient_2d.problem, mesh_family["S6"]
)

In [7]:
results = []
for name, cfg_json in tqdm.tqdm(configs.items()):
    cfg = pyamgx.Config().create_from_dict(cfg_json)
    results.append(
        {
            **test_config(
                cfg, double_precision_discrete_problem, rep_setup=1, rep_solve=1
            ),
            "config": name,
        }
    )
    cfg.destroy()

df = pd.DataFrame(results)

 94%|█████████▎| 58/62 [01:48<00:03,  1.21it/s]!!! detected some memory leaks in the code: trying to free non-empty temporary device pool !!!
ptr:     0x7372fa3f2000 size: 4096
100%|██████████| 62/62 [01:50<00:00,  1.77s/it]


In [8]:
df

,setup_time,solve_time,residual,config
0,0.033418,1.627822,NaN,AGGREGATION_DILU
1,0.074474,0.114596,2.327248e-09,AGGREGATION_GS
2,0.073881,0.395072,NaN,AGGREGATION_JACOBI
3,0.006504,0.393688,NaN,AGGREGATION_LOW_DEG_BJ
4,0.085621,1.583193,NaN,AGGREGATION_LOW_DEG_DILU
...,...,...,...,...
57,0.013310,0.026355,1.488633e-09,V-cheby-smoother
58,0.011626,0.034192,1.681975e-09,V-cheby_poly-smoother
59,0.007201,0.284652,NaN,V
60,0.007158,0.768731,NaN,W


In [9]:
working_configs = list(df[df.residual < 1e-7]["config"])
len(working_configs)

24

In [10]:
double_precision_discrete_problem = discretize(
    continuous_coefficient_2d.problem, mesh_family["S8"]
)

In [11]:
results = []
for name in tqdm.tqdm(working_configs):
    cfg_json = configs[name]
    cfg = pyamgx.Config().create_from_dict(cfg_json)
    results.append(
        {
            "config": name,
            **test_config(
                cfg, double_precision_discrete_problem, rep_setup=1, rep_solve=3
            ),
        }
    )

df2 = pd.DataFrame(results)

100%|██████████| 24/24 [00:34<00:00,  1.42s/it]


In [12]:
stats8 = df2.pivot_table(
    index="config", values=["setup_time", "solve_time", "residual"], aggfunc="min"
)
stats8

,residual,setup_time,solve_time
config,,,
AGGREGATION_GS,5.644005e-10,0.022353,1.632411
AGGREGATION_LOW_DEG_GS,5.644005e-10,0.021796,1.630934
AGGREGATION_THRUST_GS,5.181550e-10,0.019770,1.650137
AMG_AGGRREGATION_CG,4.097206e-10,0.010908,1.081007
AMG_CLASSICAL_AGGRESSIVE_CHEB_L1_TRUNC,1.890860e-10,0.025868,0.042937
AMG_CLASSICAL_AGGRESSIVE_L1_TRUNC__MOD_PCG,2.357280e-10,0.025178,0.029613
AMG_CLASSICAL_AGGRESSIVE_L1__MOD_PCG,3.166036e-10,0.023500,0.028876
AMG_CLASSICAL_L1_AGGRESSIVE_HMIS__MOD_PCG,3.683362e-10,0.730564,0.024194
AMG_CLASSICAL_L1_TRUNC__MOD_PCG,2.461980e-10,0.031142,0.026110


In [13]:
fast_solvers = list(
    stats8[
        (stats8.solve_time < 2 * stats8.solve_time.min()) & (stats8.residual < 1e-06)
    ].index
)
fast_solvers

['AMG_CLASSICAL_AGGRESSIVE_CHEB_L1_TRUNC',
 'AMG_CLASSICAL_AGGRESSIVE_L1_TRUNC__MOD_PCG',
 'AMG_CLASSICAL_AGGRESSIVE_L1__MOD_PCG',
 'AMG_CLASSICAL_L1_AGGRESSIVE_HMIS__MOD_PCG',
 'AMG_CLASSICAL_L1_TRUNC__MOD_PCG',
 'AMG_CLASSICAL_PMIS__MOD_PCG',
 'FGMRES_CLASSICAL_AGGRESSIVE_HMIS__MOD_PCG',
 'FGMRES_CLASSICAL_AGGRESSIVE_PMIS__MOD_PCG',
 'GMRES_AMG_D2__MOD_PCG']

In [14]:
# HMIS Thrust errors for large problems
fast_solvers = [solver for solver in fast_solvers if not "HMIS" in solver]
fast_solvers

['AMG_CLASSICAL_AGGRESSIVE_CHEB_L1_TRUNC',
 'AMG_CLASSICAL_AGGRESSIVE_L1_TRUNC__MOD_PCG',
 'AMG_CLASSICAL_AGGRESSIVE_L1__MOD_PCG',
 'AMG_CLASSICAL_L1_TRUNC__MOD_PCG',
 'AMG_CLASSICAL_PMIS__MOD_PCG',
 'FGMRES_CLASSICAL_AGGRESSIVE_PMIS__MOD_PCG',
 'GMRES_AMG_D2__MOD_PCG']

In [15]:
double_precision_discrete_problem = discretize(
    continuous_coefficient_2d.problem, mesh_family["S11"]
)

In [16]:
results = []
for config in tqdm.tqdm(fast_solvers):
    cfg_json = configs[config]
    cfg = pyamgx.Config().create_from_dict(cfg_json)
    results.append(
        {
            "config": config,
            **test_config(
                cfg, double_precision_discrete_problem, rep_setup=3, rep_solve=30
            ),
        }
    )

df3 = pd.DataFrame(results)

100%|██████████| 7/7 [03:54<00:00, 33.55s/it]


In [17]:
stats11 = df3.pivot_table(
    index="config", values=["setup_time", "solve_time", "residual"], aggfunc="min"
)
stats11

,residual,setup_time,solve_time
config,,,
AMG_CLASSICAL_AGGRESSIVE_CHEB_L1_TRUNC,3.314621e-11,0.786330,1.292263
AMG_CLASSICAL_AGGRESSIVE_L1_TRUNC__MOD_PCG,5.236886e-11,0.785262,0.967132
AMG_CLASSICAL_AGGRESSIVE_L1__MOD_PCG,4.163874e-11,0.775432,0.853024
AMG_CLASSICAL_L1_TRUNC__MOD_PCG,3.817748e-11,1.126447,0.857553
AMG_CLASSICAL_PMIS__MOD_PCG,4.404490e-11,0.785585,0.968356
FGMRES_CLASSICAL_AGGRESSIVE_PMIS__MOD_PCG,5.236886e-11,0.785219,0.969172
GMRES_AMG_D2__MOD_PCG,2.227760e-11,0.530277,1.065272


In [18]:
stats11.sort_values("solve_time")

,residual,setup_time,solve_time
config,,,
AMG_CLASSICAL_AGGRESSIVE_L1__MOD_PCG,4.163874e-11,0.775432,0.853024
AMG_CLASSICAL_L1_TRUNC__MOD_PCG,3.817748e-11,1.126447,0.857553
AMG_CLASSICAL_AGGRESSIVE_L1_TRUNC__MOD_PCG,5.236886e-11,0.785262,0.967132
AMG_CLASSICAL_PMIS__MOD_PCG,4.404490e-11,0.785585,0.968356
FGMRES_CLASSICAL_AGGRESSIVE_PMIS__MOD_PCG,5.236886e-11,0.785219,0.969172
GMRES_AMG_D2__MOD_PCG,2.227760e-11,0.530277,1.065272
AMG_CLASSICAL_AGGRESSIVE_CHEB_L1_TRUNC,3.314621e-11,0.786330,1.292263


In [19]:
stats11.to_csv("../results/amgx_selection_2d.csv")